In [1]:
import requests
import re
import glob, os, shutil
import gzip
import subprocess
from bs4 import BeautifulSoup
import random

### Part 3.1 Create a List of Android Apps

In [2]:
def get_submap_xmls(sitemap):
    resp = requests.get(sitemap)
    soup = BeautifulSoup(resp.content, 'xml')
    url = soup.find_all('loc') 
    result = []
    for link in url:
        result += [link.get_text()]
    return result
#extract categories list
def category(link_lst):
    result = [] 
    for xml in link_lst:
        result += [re.search('(?<=sitemaps\/)(.*)(?=\-\d)|(?<=sitemaps\/)(.*)(?=\.xml)',xml).groups()[1]]
    return [i for i in result if i] 


#get all the gz files from each categories
def sample_from_cat(categories): 
    soups = []
    for c in categories:
        url = 'https://apkpure.com/sitemaps/{}.xml.gz'.format(c)
        try:
            r = requests.get(url)
        except:
            url = 'https://apkpure.com/sitemaps/{}.xml.gz'.format(c+'-1')
            r = requests.get(url)
        #decompress the gz file and parse xml
        data = gzip.decompress(r.content)
        soup = BeautifulSoup(data,features = 'lxml')
        soups.append(soup)
    return soups

def get_app_urls(sitemap,cat,number):
    xmls = get_submap_xmls(sitemap)
    
    if cat == 'all':
        categories = category(xmls)
    elif type(cat) == int:
        categories = random.choices(category(xmls), k = cat)
    else:
        categories = cat
        
    soups = sample_from_cat(categories)
    apps = []
    for soup in soups:
        count = 0
        sp = soup.find_all(re.compile('loc')) 
        lst = [] 
        for i in sp:
            if re.match('<loc>', str(i)) and count < number:
                try:
                    lst += [re.search('(?<=<loc>)(https:\/\/apkpure.com\/.*?\/.*[a-zA-Z\d].*)(?=<\/loc>)', str(i)).group()] #find all urls storec in loc
                    count += 1
                except:
                    continue
        apps += lst
    return apps

In [67]:
def download_apk(outpath, app_urls, cat):
    if not os.path.exists(outpath):
        os.mkdir(outpath)
    if not os.path.exists(outpath + '/' + cat):
        os.mkdir(outpath + '/' + cat)

    for url in app_urls:
        download_link = url + '/download?from=details'
        r1 = requests.get(download_link)
        soup = BeautifulSoup(r1.text)
        try:
            download_link_file = soup.find('div',attrs = {"class":"fast-download-box fast-bottom"}).find('p',attrs = {'class':'down-click'}).find('a',href = True)['href']
        except:
            continue
        r2 = requests.get(download_link_file)
        apkfile = r2.content
        complete_name = os.path.join(outpath+'/'+cat+'/', url.split('/')[-1]+".apk")
        out_name = os.path.join(outpath+'/'+cat+'/', url.split('/')[-1])
        with open(complete_name, 'wb') as fh:
            fh.write(apkfile)
        subprocess.call(['apktool', 'd', outpath+'/'+cat+'/'+url.split('/')[-1]+".apk", '-o',out_name])

In [68]:
process_apk('data', apps[:20], 'dating')

In [69]:
process_apk('data', apps[20:], 'education')

In [76]:
clean_disk('data')

In [10]:
app_link = apps[0]
outpath='data'
app_name = app_link.split('/')[-1]
    

In [17]:
download_apk(app_link, outpath)

In [18]:
decompile(app_link, outpath, 'dating')

In [11]:
#get download link
download_link = app_link + '/download?from=details'
r = requests.get(download_link)
apkfile = r.content

In [13]:
#download an save in outpath directory
complete_name = outpath+'/'+app_name+'.apk'
with open(complete_name, 'wb') as f:
    f.write(apkfile)

In [15]:
decompile(apps[0], 'data','dating')

### 3.2 Download and Decompile apk Files

In [50]:
def download_apk(app_link, outpath, cat):
    '''
    From the app link, find the download page, obtain the download link
    '''
    name = app_link.split('/')[-1]
    
    r = requests.get(app_link)
    apk = r.content
    
    if not os.path.exists(outpath):
        os.mkdir(outpath)
    
    if not os.path.exists(outpath + '/' + cat):
        os.mkdir(outpath + '/' + cat)
            
    with open(outpath + '/' + cat + '/' + name + '.apk', 'wb') as f:
        f.write(apk)

In [56]:
def decompile(app_link, outpath, cat):
    '''
    download apk file and decompile to local path
    '''
    if not os.path.exists(outpath):
        os.mkdir(outpath)
    
    if not os.path.exists(outpath + '/' + cat):
        os.mkdir(outpath + '/' + cat)
    name = app_link.split('/')[-1]
    original = outpath + '/' + cat + '/' + name + '.apk'
    decomp = outpath + '/' + cat + '/' + name
    subprocess.call(['apktool', 'd', original, '-o', decomp])

In [57]:
def get_smali_code(app_urls, outpath, cat): #download and decompile all application urls
    for url in app_urls:
        try:
            download_apk(url, outpath, cat)
            decompile(url, outpath, cat)
        except:
            continue

In [6]:
apps = get_app_urls('https://apkpure.com/sitemap.xml',['dating','education'],20)

In [58]:
get_smali_code(apps[:20], 'data', 'dating')

In [62]:
get_smali_code(apps[20:], 'data', 'education')

### 3.3 Organize Disk

In [74]:
#use for d, dirs, files in os.walk(root):
def clean_app_folder(app_path):
    if '.DS_Store' not in app_path:
        if os.path.isdir(app_path):
            subs = os.listdir(app_path)
            for s in subs:
                if s not in ['smali', 'AndroidManifest.xml']:
                    path = app_path+'/'+s
                    if os.path.isdir(path):
                        shutil.rmtree(path)
                    elif os.path.isfile(path):
                        os.remove(path)
        else:
            os.remove(app_path)

In [75]:
def clean_disk(outpath):
    folders = os.listdir(outpath)
    for f in folders:
        if os.path.isdir(outpath+'/'+f):
            apps = os.listdir(outpath+'/'+f)
            for app in apps:
                clean_app_folder(outpath+'/'+f+'/'+app)
        else:
            os.remove(outpath+'/'+f)